In [17]:
import numpy as np
import pymc3 as pm
import pandas as pd

from aesara import tensor as at

In [18]:
sunspot_df = pd.read_csv(
    pm.get_data("sunspot.csv"), sep=";", names=["time", "sunspot.year"], usecols=[0, 1]
)
sunspot_df.head()

,time,sunspot.year
0,1700.5,8.3
1,1701.5,18.3
2,1702.5,26.7
3,1703.5,38.3
4,1704.5,60.0


In [19]:
def mixture_logp(weights, comp_dist, atoms, obs):    
    
    # comp_dist is something like pm.Normal.logp
    
    atoms_shape = atoms.shape
    obs_shape = obs.shape
    
    assert atoms.shape == weights.shape
    
    
    
    weights = np.broadcast_to(weights, shape=obs_shape + atoms_shape)
    log_weights = (np.log(weights)).T
    atoms_logp = comp_dist(
        np.broadcast_to(obs, atoms_shape + obs_shape), 
        np.broadcast_to(atoms, obs_shape + atoms_shape).T, 
    )
    
    return np.exp(log_weights + atoms_logp).sum(axis=0)

In [20]:
alpha = [1, 1, 1,]; K = 19

In [21]:
weights = np.random.dirichlet(alpha=alpha, size=[]) # placeholder
atoms = np.random.normal(size=[len(alpha),])
observations = np.array([-1, -0.5, 0, 0.5, 1])

mixture_logp(
    weights=weights,
    comp_dist=lambda obs, atoms: pm.Normal.logp(obs, atoms, 1).eval(),
    atoms=atoms,
    obs=observations,
)

array([0.29188662, 0.36723628, 0.36842861, 0.2955007 , 0.18996207])

In [22]:
def stick_breaking(beta):
    portion_remaining = at.concatenate([[1], at.cumprod(1 - beta)[:-1]])

    return beta * portion_remaining

In [29]:
with pm.Model() as ss_model:
    
    α = pm.Gamma('α', 1., 1.)
    β = pm.Beta('β', 1, α, shape=K)
    w = pm.Deterministic('w', stick_breaking(β))
    
    μ = pm.Uniform('μ', 0., 300., shape=K)
    
    # line below
    obs = pm.Potential('obs', w, pm.Poisson.dist(μ), observed=sunspot_df["sunspot.year"].values)

TypeError: Potential() got an unexpected keyword argument 'observed'

In [32]:
with pm.Model() as ss_model:
    
    α = pm.Gamma('α', 1., 1.)
    β = pm.Beta('β', 1, α, shape=K)
    w = pm.Deterministic('w', stick_breaking(β))
    
    μ = pm.Uniform('μ', 0., 300., shape=K)
    obs = pm.Mixture('obs', w, pm.Poisson.dist(μ), observed=sunspot_df["sunspot.year"].values)

TypeError: dist() takes 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given